In [ ]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.applications import MobileNetV2
from sklearn.model_selection import train_test_split
from google.colab import drive
drive.mount('/content/drive')
import os
drive_path='/content/drive/Mydrive/'


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:


# Define the dataset path
base_dir = '/content/drive/MyDrive/dataset'
# Define parameters for image processing
img_height, img_width = 224, 224  # Image size for input
batch_size = 32

# Use ImageDataGenerator to load and augment images
train_datagen = ImageDataGenerator(
    rescale=1.0/255,          # Normalize pixel values to [0, 1]
    rotation_range=30,        # Randomly rotate images
    width_shift_range=0.2,    # Shift images horizontally
    height_shift_range=0.2,   # Shift images vertically
    shear_range=0.2,          # Apply shear transformation
    zoom_range=0.2,           # Apply zoom
    horizontal_flip=True,     # Flip images horizontally
    validation_split=0.2      # Use 20% of the data for validation
)

# Create training and validation generators
train_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'  # Use training subset
)

val_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'  # Use validation subset
)

# Verify class indices
print("Class Indices:", train_generator.class_indices)

Found 1224 images belonging to 6 classes.
Found 306 images belonging to 6 classes.
Class Indices: {'alien_test': 0, 'cloudy': 1, 'foggy': 2, 'rainy': 3, 'shine': 4, 'sunrise': 5}


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from collections import Counter

# Define the dataset path
base_dir = '/content/drive/MyDrive/dataset'

# Define parameters for image processing
img_height, img_width = 224, 224  # Image size for input
batch_size = 32

# Data Augmentation and Preprocessing
train_datagen = ImageDataGenerator(
    rescale=1.0/255,          # Normalize pixel values to [0, 1]
    rotation_range=30,        # Randomly rotate images
    width_shift_range=0.2,    # Shift images horizontally
    height_shift_range=0.2,   # Shift images vertically
    shear_range=0.2,          # Apply shear transformation
    zoom_range=0.2,           # Apply zoom
    horizontal_flip=True,     # Flip images horizontally
    validation_split=0.2      # Use 20% of the data for validation
)

# Training and Validation Generators
train_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'  # Use training subset
)

val_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'  # Use validation subset
)

# Check class indices
print("Class Indices:", train_generator.class_indices)

# Define the CNN Model
model = Sequential([
    Input(shape=(img_height, img_width, 3)),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(train_generator.class_indices), activation='softmax')  # Adjust output layer for the number of classes
])

# Compile the Model
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Define Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_weather_model.keras', save_best_only=True, monitor='val_loss', mode='min')

# Train the Model
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=50,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_steps=val_generator.samples // val_generator.batch_size,
    callbacks=[early_stopping, model_checkpoint]
)

# Save the final model
model.save('final_weather_classification_model.keras')

# Evaluate the Model
loss, accuracy = model.evaluate(val_generator)
print(f"Validation Accuracy: {accuracy * 100:.2f}%")

# Make Predictions
import numpy as np

# Predict on validation set (example)
sample_images, sample_labels = next(val_generator)
predictions = model.predict(sample_images)

# Decode Predictions
predicted_classes = np.argmax(predictions, axis=1)
actual_classes = np.argmax(sample_labels, axis=1)

print(f"Predicted Classes: {predicted_classes}")
print(f"Actual Classes: {actual_classes}")

Found 1224 images belonging to 6 classes.
Found 306 images belonging to 6 classes.
Class Indices: {'alien_test': 0, 'cloudy': 1, 'foggy': 2, 'rainy': 3, 'shine': 4, 'sunrise': 5}
Epoch 1/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 221s 6s/step - accuracy: 0.3913 - loss: 1.6969 - val_accuracy: 0.6076 - val_loss: 1.3149
Epoch 2/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 5s 39ms/step - accuracy: 0.6250 - loss: 1.4739 - val_accuracy: 0.6111 - val_loss: 1.2011
Epoch 3/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 216s 5s/step - accuracy: 0.5707 - loss: 1.1263 - val_accuracy: 0.6146 - val_loss: 1.1347
Epoch 4/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 5s 40ms/step - accuracy: 0.5312 - loss: 1.1415 - val_accuracy: 0.5556 - val_loss: 1.0136
Epoch 5/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 269s 6s/step - accuracy: 0.6457 - loss: 0.9185 - val_accuracy: 0.6806 - val_loss: 0.8856
Epoch 6/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step - accuracy: 0.7188 - loss: 0.9111 - val_accuracy: 0.5556 - val_loss: 1.0751
Epoch 7/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 241s 5s/step - accura

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from collections import Counter

# Define the dataset path
base_dir = '/content/drive/MyDrive/dataset'

# Define parameters for image processing
img_height, img_width = 224, 224
batch_size = 32

# Data Augmentation and Preprocessing
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    brightness_range=[0.5, 1.5],  # Adjust brightness
    horizontal_flip=True,
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

# Check class distribution and calculate class weights
class_counts = Counter(train_generator.classes)
class_weights = {i: max(class_counts.values()) / count for i, count in class_counts.items()}
print("Class Weights:", class_weights)

# Define an Improved CNN Model
model = Sequential([
    Input(shape=(img_height, img_width, 3)),
    Conv2D(32, (3, 3), activation='relu'),
    BatchNormalization(),  # Normalize activations
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(256, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),  # Reduce overfitting
    Dense(len(train_generator.class_indices), activation='softmax')  # Output layer
])

# Compile the Model with Learning Rate Scheduler
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Define Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_weather_model_improved.keras', save_best_only=True, monitor='val_loss', mode='min')
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)

# Train the Model
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=50,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_steps=val_generator.samples // val_generator.batch_size,
    class_weight=class_weights,  # Handle imbalanced classes
    callbacks=[early_stopping, model_checkpoint, reduce_lr]
)

# Save the final model
model.save('final_weather_classification_model_improved.keras')

# Evaluate the Model
loss, accuracy = model.evaluate(val_generator)
print(f"Validation Accuracy: {accuracy * 100:.2f}%")

# Make Predictions
import numpy as np

# Predict on validation set (example)
sample_images, sample_labels = next(val_generator)
predictions = model.predict(sample_images)

# Decode Predictions
predicted_classes = np.argmax(predictions, axis=1)
actual_classes = np.argmax(sample_labels, axis=1)

print(f"Predicted Classes: {predicted_classes}")
print(f"Actual Classes: {actual_classes}")

Found 1224 images belonging to 6 classes.
Found 306 images belonging to 6 classes.
Class Weights: {0: 11.666666666666666, 1: 1.1666666666666667, 2: 1.1666666666666667, 3: 1.1666666666666667, 4: 1.4, 5: 1.0}
Epoch 1/50


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


38/38 ━━━━━━━━━━━━━━━━━━━━ 616s 14s/step - accuracy: 0.4448 - loss: 24.0305 - val_accuracy: 0.1944 - val_loss: 11.7483 - learning_rate: 0.0010
Epoch 2/50
 1/38 ━━━━━━━━━━━━━━━━━━━━ 3:47 6s/step - accuracy: 0.5625 - loss: 11.8863

/usr/lib/python3.11/contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


38/38 ━━━━━━━━━━━━━━━━━━━━ 8s 38ms/step - accuracy: 0.5625 - loss: 11.8863 - val_accuracy: 0.3333 - val_loss: 8.6557 - learning_rate: 0.0010
Epoch 3/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 285s 7s/step - accuracy: 0.4754 - loss: 32.3340 - val_accuracy: 0.2153 - val_loss: 14.3871 - learning_rate: 0.0010
Epoch 4/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 6s 17ms/step - accuracy: 0.3438 - loss: 22.5178 - val_accuracy: 0.3889 - val_loss: 14.5146 - learning_rate: 0.0010
Epoch 5/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 270s 7s/step - accuracy: 0.4563 - loss: 37.4952 - val_accuracy: 0.2222 - val_loss: 21.7259 - learning_rate: 0.0010
Epoch 6/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 23s 449ms/step - accuracy: 0.3750 - loss: 21.5459 - val_accuracy: 0.1667 - val_loss: 25.3533 - learning_rate: 5.0000e-04
Epoch 7/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 307s 7s/step - accuracy: 0.4828 - loss: 29.4962 - val_accuracy: 0.2431 - val_loss: 9.6098 - learning_rate: 5.0000e-04
10/10 ━━━━━━━━━━━━━━━━━━━━ 19s 2s/step - accuracy: 0.2291 - loss: 10.7783
Validation 

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.applications import ResNet50
from collections import Counter

# Define the dataset path
base_dir = '/content/drive/MyDrive/dataset'

# Define parameters for image processing
img_height, img_width = 224, 224
batch_size = 32

# Data Augmentation and Preprocessing
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    brightness_range=[0.5, 1.5],  # Added brightness adjustment
    horizontal_flip=True,
    validation_split=0.2  # 20% for validation
)

train_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

# Check class distribution and calculate class weights
class_counts = Counter(train_generator.classes)
class_weights = {i: max(class_counts.values()) / count for i, count in class_counts.items()}
print("Class Weights:", class_weights)

# Load the ResNet50 model for transfer learning
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))
base_model.trainable = False  # Freeze the base model layers

# Build the model
model = Sequential([
    base_model,
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(train_generator.class_indices), activation='softmax')  # Output layer
])

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Define callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_weather_model_resnet.keras', save_best_only=True, monitor='val_loss', mode='min')
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)

# Train the model
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=50,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_steps=val_generator.samples // val_generator.batch_size,
    class_weight=class_weights,  # Apply class weights
    callbacks=[early_stopping, model_checkpoint, reduce_lr]
)

# Save the final model
model.save('final_weather_classification_model_resnet.keras')

# Evaluate the model
loss, accuracy = model.evaluate(val_generator)
print(f"Validation Accuracy: {accuracy * 100:.2f}%")

# Make Predictions (example on validation data)
import numpy as np

sample_images, sample_labels = next(val_generator)
predictions = model.predict(sample_images)

# Decode Predictions
predicted_classes = np.argmax(predictions, axis=1)
actual_classes = np.argmax(sample_labels, axis=1)

print(f"Predicted Classes: {predicted_classes}")
print(f"Actual Classes: {actual_classes}")

Found 1224 images belonging to 6 classes.
Found 306 images belonging to 6 classes.
Class Weights: {0: 11.666666666666666, 1: 1.1666666666666667, 2: 1.1666666666666667, 3: 1.1666666666666667, 4: 1.4, 5: 1.0}
94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
Epoch 1/50


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


38/38 ━━━━━━━━━━━━━━━━━━━━ 749s 17s/step - accuracy: 0.2053 - loss: 10.0087 - val_accuracy: 0.1910 - val_loss: 1.8533 - learning_rate: 0.0010
Epoch 2/50
 1/38 ━━━━━━━━━━━━━━━━━━━━ 3:37 6s/step - accuracy: 0.3125 - loss: 2.1871

/usr/lib/python3.11/contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


38/38 ━━━━━━━━━━━━━━━━━━━━ 24s 484ms/step - accuracy: 0.3125 - loss: 2.1871 - val_accuracy: 0.1667 - val_loss: 1.7921 - learning_rate: 0.0010
Epoch 3/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 362s 9s/step - accuracy: 0.2027 - loss: 2.4666 - val_accuracy: 0.0174 - val_loss: 1.7924 - learning_rate: 0.0010
Epoch 4/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 15s 166ms/step - accuracy: 0.0000e+00 - loss: 2.1107 - val_accuracy: 0.0556 - val_loss: 1.7919 - learning_rate: 0.0010
Epoch 5/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 371s 10s/step - accuracy: 0.0263 - loss: 2.5597 - val_accuracy: 0.0208 - val_loss: 1.7938 - learning_rate: 0.0010
Epoch 6/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 9s 84ms/step - accuracy: 0.0312 - loss: 2.6419 - val_accuracy: 0.0000e+00 - val_loss: 1.7941 - learning_rate: 0.0010
Epoch 7/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 357s 9s/step - accuracy: 0.0228 - loss: 2.5255 - val_accuracy: 0.0208 - val_loss: 1.7936 - learning_rate: 0.0010
Epoch 8/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 12s 81ms/step - accuracy: 0.0000e+00 - loss: 2.0607 - 

In [3]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np
import math
import warnings

# Suppress specific warnings
warnings.filterwarnings("ignore", category=UserWarning, module="keras.src.trainers.data_adapters.py_dataset_adapter")

# Define the dataset path
base_dir = '/content/drive/MyDrive/dataset'

# Define parameters for image processing
img_height, img_width = 224, 224
batch_size = 8 # Adjust batch size for better handling of smaller datasets

# Data Augmentation and Preprocessing
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    brightness_range=[0.5, 1.5],
    horizontal_flip=True,
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

# Calculate steps per epoch
steps_per_epoch = train_generator.n // train_generator.batch_size
validation_steps = val_generator.n // val_generator.batch_size

# Define a CNN Model with Batch Normalization
model = Sequential([
    Input(shape=(img_height, img_width, 3)),
    Conv2D(32, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(256, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(train_generator.class_indices), activation='softmax')
])

# Compile the Model
model.compile(optimizer=Adam(learning_rate=1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Define Callbacks
model_checkpoint = ModelCheckpoint('best_weather_model.keras', save_best_only=True, monitor='val_loss', mode='min')
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)

# Train the Model
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=50,
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps,
    callbacks=[model_checkpoint, reduce_lr],  # Simplified callback list
    #workers=4,
    #use_multiprocessing=True
)

# Save the Final Model
model.save('final_weather_classification_model.keras')

# Evaluate the Model
loss, accuracy = model.evaluate(val_generator, steps=validation_steps)
print(f"Validation Accuracy: {accuracy * 100:.2f}%")

# Predict on Validation Data
val_predictions = model.predict(val_generator, steps=validation_steps)
y_pred = np.argmax(val_predictions, axis=1)
y_true = val_generator.classes

# Confusion Matrix and Classification Report
print("\nConfusion Matrix:")
print(confusion_matrix(y_true, y_pred))

print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=list(train_generator.class_indices.keys())))

# Predict on Sample Batch.
sample_images, sample_labels = next(val_generator)
predictions = model.predict(sample_images)

# Decode Predictions
predicted_classes = np.argmax(predictions, axis=1)
actual_classes = np.argmax(sample_labels, axis=1)

print(f"Predicted Classes: {predicted_classes}")
print(f"Actual Classes: {actual_classes}")

Found 1224 images belonging to 6 classes.
Found 306 images belonging to 6 classes.
Epoch 1/50
153/153 ━━━━━━━━━━━━━━━━━━━━ 313s 2s/step - accuracy: 0.4845 - loss: 3.0441 - val_accuracy: 0.2204 - val_loss: 4.1542 - learning_rate: 1.0000e-04
Epoch 2/50


/usr/lib/python3.11/contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


153/153 ━━━━━━━━━━━━━━━━━━━━ 20s 130ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 6.1762 - learning_rate: 1.0000e-04
Epoch 3/50
153/153 ━━━━━━━━━━━━━━━━━━━━ 296s 2s/step - accuracy: 0.6066 - loss: 1.7450 - val_accuracy: 0.4046 - val_loss: 3.3382 - learning_rate: 1.0000e-04
Epoch 4/50
153/153 ━━━━━━━━━━━━━━━━━━━━ 21s 136ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.5000 - val_loss: 3.2799 - learning_rate: 1.0000e-04
Epoch 5/50
153/153 ━━━━━━━━━━━━━━━━━━━━ 311s 2s/step - accuracy: 0.5928 - loss: 1.3714 - val_accuracy: 0.5329 - val_loss: 1.6046 - learning_rate: 1.0000e-04
Epoch 6/50
153/153 ━━━━━━━━━━━━━━━━━━━━ 21s 139ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0222 - learning_rate: 1.0000e-04
Epoch 7/50
153/153 ━━━━━━━━━━━━━━━━━━━━ 298s 2s/step - accuracy: 0.6586 - loss: 1.2279 - val_accuracy: 0.6743 - val_loss: 0.9608 - learning_rate: 1.0000e-04
Epoch 8/50
153/153 ━━━━━━━━━━━━━━━━

ValueError: Found input variables with inconsistent numbers of samples: [306, 304]